In [1]:
# 全局设置
import os
import datetime as dt

import numpy as np
import pandas as pd

import QuantStudio.api as QS
fd = QS.FactorDB.FactorTools
Factorize = QS.FactorDB.Factorize

# 因子定义框架

所有的因子划分成两类:
* **基础因子**: 指直接由原始数据转化而来, 并不依赖于其他因子的因子, 比如最新股东权益因子就是以财务报表里的股东权益合计这个数据项转化而来的基础因子, 大多数的基础因子通过因子表的 getFactor() 方法得到
* **衍生因子**: 依赖于其他因子通过因子运算得到的因子, 而 BP 因子则是用最新股东权益因子除以总市值因子运算得来的衍生因子.

因子的运算分成四类: **单点运算**, **时间序列运算**, **横截面运算**以及**面板运算**. 每种运算的适用范围各不相同, 在效率上也有所差异. 这些运算可以嵌套组合以形成更复杂的计算.

因子定义逻辑上是一个有向无环的计算图(DAG). 对于已有的因子可以施加上述四种因子运算生成新因子, 新生成的因子跟已有的因子没有区别, 可以继续施加因子运算生成其他因子, 该定义的层次数目没有限制, 可以一直叠加上去.

以 HSIGMA 因子的定义举例说明. HSIGMA 因子是 Barra 中国市场风险模型(CNE5)里的一个风险因子, 称为历史残余波动率因子. 该因子的计算首先用股票的超额收益率和市场超额收益率进行时间序列回归得到残余收益率因子 EPSILON, 然后取一段时间的残余收益率求其标准差得到残余波动率因子 HSIGMA. 在 Quant Studio 中, HSIGMA 因子是由 EPSILON 因子通过时间序列运算生成, 而 EPSILON 由 BETA 因子、市场收益率因子、股票收益率因子再通过时间序列运算生成, 最后, BETA 因子则是基于股票收益率因子和市场收益率因子计算得到, 而这些收益率因子则是由价格等基础因子通过施加单点运算得到. 整个的因子定义共有六层, 最底层是基础因子, 而其他层是由已经定义好的衍生因子或者基础因子组成, 因子可以复用, 在任意层的运算里都可以引用已经定义过的任何因子, 具体定义如下图所示:

![因子定义模型](../images/因子定义模型.png)

# 因子运算

In [2]:
# 示例因子库
# FDB = QS.FactorDB.HDF5DB(sys_args={"主目录": "../Data/HDF5"}).connect();
FDB = QS.FactorDB.HDF5DB(config_file="../config/HDF5DBConfig.json").connect();

## 单点运算

固定时点、固定 ID 的不同因子间的运算. 单点运算是最简单的一类因子运算, 其典型例子是各种估值因子的定义, 比如市净率(BP)因子, 其是由股东权益除以总市值得到. BP 因子的描述子即为股东权益因子和总市值因子, 而定义算子是一个除法运算(真正的实现可能还要包括缺失值处理以及对于分母为 0 的处理).

单点运算衍生因子由 PointOperation 定义，其 `__init__` 方法参数:	
* name: str, 因子名称
* descriptors: list(Factor), 该因子依赖的因子列表
* sys_args: dict, 因子对象参数集

因子对象参数集:
* 算子: function, 用户自定义的运算函数, 输入参数必须为: (f, idt, iid, x, args)
* 参数: dict, 用户自定义的传递给算子的附加参数, 即算子的最后一个输入参数 args.
* 数据类型: str, 可选 "double"(默认值), "string", "object", 因子的数据类型, 该参数也指明了算子的返回值类型
* 运算时点: str, 可选 "单时点"(默认值), "多时点", 传入算子进行运算的时点是单个时点还是一列时点, 通常可以在时间维度向量化运算的情况下选择 "多时点".
* 运算ID: str, 可选: "单ID"(默认值), "多ID", 传入算子进行运算的 ID 是单个 ID 还是一列 ID, 通常可以在 ID 维度向量化运算的情况下选择 "多ID"。

参数行为:
* "运算时点" 为 "单时点", "运算ID" 为 "单ID":
    * f: Factor, 该算子隶属于的因子对象, 即调用该算子的因子对象
    * idt: datetime, 当前待计算的时点
    * iid: str, 当前待计算的 ID
    * x: list(标量), 定义该因子所依赖因子的数据, x[i] 为第 i 个描述子在时点 idt, ID 为 iid 时的数据
    * args: dict, 用户传入的自定义参数
    * 返回值: 标量, 给定时点 idt 给定 ID iid 的衍生因子值
* "运算时点" 为 "多时点", "运算ID" 为 "单ID":
    * f: Factor, 该算子隶属于的因子对象, 即调用该算子的因子对象
    * idt: list(datetime), 当前一起计算的时间序列, 注意该序列并不一定为全体运算时点
    * iid: str, 当前待计算的 ID
    * x: list(array), 定义该因子所依赖因子的数据, x[i] 为第 i 个描述子在 ID 为 iid 时的因子数据: array(shape=(len(idt), ))
    * args: dict, 用户传入的自定义参数
    * 返回值: array(shape=(len(idt), )), 衍生因子值
* "运算时点" 为 "单时点", "运算ID" 为 "多ID":
    * f: Factor, 该算子隶属于的因子对象, 即调用该算子的因子对象
    * idt: datetime, 当前待计算的时点
    * iid: list(str), 当前一起计算的 ID 序列, 注意该序列并不一定为整个截面
    * x: list(array), 定义该因子所依赖因子的数据, x[i] 为第 i 个描述子在时点为 idt 时的因子数据: array(shape=(len(iid), ))
    * args: dict, 用户传入的自定义参数
    * 返回值: array(shape=(len(iid), )), 衍生因子值
* "运算时点" 为 "多时点", "运算ID" 为 "多ID":
    * f: Factor, 该算子隶属于的因子对象, 即调用该算子的因子对象
    * idt: list(datetime), 当前一起计算的时间序列, 注意该序列并不一定为全体运算时点
    * iid: list(str), 当前一起计算的 ID 序列, 注意该序列并不一定为整个截面
    * x: list(array), 定义该因子所依赖因子的数据, x[i] 为第 i 个描述子的因子数据: array(shape=(len(idt), len(iid)))
    * args: dict, 用户传入的自定义参数
    * 返回值: array(shape=(len(idt), len(iid))), 衍生因子值

In [3]:
# 单点运算
FT = FDB.getTable(table_name="stock_cn_day_bar_nafilled")
High, Low = FT.getFactor("high"), FT.getFactor("low")

def PointFun(f, idt, iid, x, args):
    print(f"idt : {idt},", f"iid : {iid},", f"args: {args},", f"x : {x}", sep="    ")
    print("======================================================")
    return (x[0] + x[1]) / 2

Args = {
    "算子": PointFun, 
    "参数": {}, 
    "数据类型": "double",
    "运算时点": "多时点", 
    "运算ID": "多ID"
}
PointFactor = QS.FactorDB.PointOperation(name="PointFactor", 
                                         descriptors=[High, Low], 
                                         sys_args=Args)

IDs = ["000001.SZ", "000002.SZ", "000003.SZ"]
DTs = FT.getDateTime(start_dt=dt.datetime(2018,11,5), end_dt=dt.datetime(2018,11,9))
print(Args)
print("======================================================")
print(PointFactor.readData(ids=IDs, dts=DTs))

{'算子': <function PointFun at 0x000001E9FEA751B0>, '参数': {}, '数据类型': 'double', '运算时点': '多时点', '运算ID': '多ID'}
idt : [datetime.datetime(2018, 11, 5, 0, 0), datetime.datetime(2018, 11, 6, 0, 0), datetime.datetime(2018, 11, 7, 0, 0), datetime.datetime(2018, 11, 8, 0, 0), datetime.datetime(2018, 11, 9, 0, 0)],    iid : ['000001.SZ', '000002.SZ', '000003.SZ'],    args: {},    x : [array([[11.04, 24.48,   nan],
       [10.93, 24.4 ,   nan],
       [10.95, 24.5 ,   nan],
       [10.97, 24.59,   nan],
       [10.73, 24.33,   nan]]), array([[10.82, 23.7 ,   nan],
       [10.73, 23.71,   nan],
       [10.75, 23.71,   nan],
       [10.82, 23.92,   nan],
       [10.4 , 23.53,   nan]])]
            000001.SZ  000002.SZ  000003.SZ
2018-11-05     10.930     24.090        NaN
2018-11-06     10.830     24.055        NaN
2018-11-07     10.850     24.105        NaN
2018-11-08     10.895     24.255        NaN
2018-11-09     10.565     23.930        NaN


## 时间序列运算

固定 ID 不同因子间在时间序列上的运算. 除了指定描述子和定义算子外, 时间序列运算还需要指定每个描述子的回溯期数. 时间序列运算的典型例子是移动平均线的定义, 其是由证券过去一段时间的价格序列取某种形式的平均得到的. 移动平均线因子的描述子即为价格因子, 而定义算子是一个平均值运算.

时间序列运算衍生因子由 TimeOperation 定义，其 `__init__` 方法参数:	
* name: str, 因子名称
* descriptors: list(Factor), 该因子依赖的因子列表
* sys_args: dict, 因子对象参数集

因子对象参数集:
* 算子: function, 用户自定义的运算函数, 输入参数必须为: (f, idt, iid, x, args)
* 参数: dict, 用户自定义的传递给算子的附加参数, 即算子的最后一个输入参数 args.
* 数据类型: str, 可选 "double"(默认值), "string", "object", 因子的数据类型, 该参数也指明了自定义算子的返回值类型
* 运算时点: str, 可选 "单时点"(默认值), "多时点", 传入算子进行运算的时点是单个时点还是一列时点, 通常可以在时间维度向量化运算的情况下选择 "多时点"
* 运算ID: str, 可选: "单ID"(默认值), "多ID", 传入算子进行运算的 ID 是单个 ID 还是一列 ID, 通常可以在 ID 维度向量化运算的情况下选择 "多ID"
* 回溯期数: list(int>=0), 每个描述子的回溯期数(不包括当前所在的时点), 比如计算 5 日平均, 则回溯期数为 5-1=4
* 回溯模式: list(str, 可选 "滚动窗口"(默认值), "扩张窗口"), 每个描述子的回溯窗口模式
* 自身回溯期数: int>=0, 默认值 0, 该因子本身需要回溯的期数, 0 表示无自身迭代, 并忽略 "自身回溯模式" 和 "自身初始值" 参数
* 自身回溯模式: str, 可选 "滚动窗口"(默认值), "扩张窗口", 该因子本身的回溯窗口模式
* 自身初始值: DataFrame(index=[时点], columns=[ID]), 自身回溯的初始值

在定义时间序列运算衍生因子时附加的参数 "运算时点", "运算ID", "回溯模式", "自身回溯模式", "自身回溯期数", "自身初始值" 取值会影响该算子的输入参数和返回值, 用户定义该算子时必须遵守下面的约定:

### 无自身迭代, 滚动窗口模式

**无自身迭代, 滚动窗口模式**: "自身回溯期数" 为 0 (无自身迭代), 所有描述子的 "回溯模式" 均为 "滚动窗口":
* "运算时点" 为 "单时点", "运算ID" 为 "单ID":
    * f: Factor, 该算子隶属于的因子对象, 即调用该算子的因子对象
    * idt: list(datetime), 当前回溯期数最大的描述子的时间序列
    * iid: str, 当前待计算的 ID
    * x: list(array(shape=(第 i 个描述子的回溯期数+1, ))), 描述子数据, x[i]: 第 i 个描述子在时点为 idt[-1], ID 为 iid 时的回溯数据, +1 表示会把当前时点的数据也添加进去
    * args: dict, 用户传入的自定义参数
    * 返回值: 标量, 给定时点 idt[-1], 给定 ID iid 的衍生因子值
* "运算时点" 为 "单时点", "运算ID" 为 "多ID":
    * f: Factor, 该算子隶属于的因子对象, 即调用该算子的因子对象
    * idt: list(datetime), 当前回溯期数最大的描述子的时间序列
    * iid: list(str), 当前一起计算的 ID 序列, 注意该序列并不一定为整个截面
    * x: list(array(shape=(第 i 个描述子的回溯期数+1, len(iid)))), 描述子数据, x[i] 为第 i 个描述子在时点为 idt[-1] 的回溯数据, +1 表示会把当前时点的数据也添加进去
    * args: dict, 用户传入的自定义参数
    * 返回值: array(shape=(len(iid), )), 衍生因子值
* "运算时点" 为 "多时点", "运算ID" 为 "单ID":
    * f: Factor, 该算子隶属于的因子对象, 即调用该算子的因子对象
    * idt: list(datetime), 注意该序列并不一定为全体运算时点, 其长度等于最长回溯期数 + 待计算的时间序列长度
    * iid: str, 当前待计算的 ID
    * x: list(array(shape=(第 i 个描述子的回溯期数+待计算的时间序列长度, ))), 描述子数据, x[i]: 第 i 个描述子在 ID 为 iid 时的因子数据 
    * args: dict, 用户传入的自定义参数
    * 返回值: array(shape=(待计算的时间序列长度, )), 衍生因子值
* "运算时点" 为 "多时点", "运算ID" 为 "多ID":
    * f: Factor, 该算子隶属于的因子对象, 即调用该算子的因子对象
    * idt: list(datetime), 注意该序列并不一定为全体运算时点, 其长度等于最长回溯期数 + 待计算的时间序列长度
    * iid: list(str), 当前一起计算的 ID 序列, 注意该序列并不一定为整个截面
    * x: list(array(shape=(第 i 个描述子的回溯期数+待计算的时间序列长度, len(iid)))), 描述子数据, x[i] 为第 i 个描述子的因子数据
    * args: dict, 用户传入的自定义参数
    * 返回值: array(shape=(待计算的时间序列长度, len(iid))), 衍生因子值

In [4]:
# 时间序列运算: 无自身迭代, 滚动窗口模式
FT = FDB.getTable(table_name="stock_cn_day_bar_nafilled")
High, Low = FT.getFactor("high"), FT.getFactor("low")

def TimeFun(f, idt, iid, x, args):
    print(f"idt : {idt},", f"iid : {iid},", f"args: {args},", f"x : {x}", sep="    ")
    print("======================================================")
    Mid = (x[0] + x[1]) / 2
    # 单时点
    # return np.nanmean(Mid)
    # 多时点, 单ID
#     w = f.Args["回溯期数"][0]
#     return pd.Series(Mid).rolling(window=w+1).mean().values[w:]
    # 多时点, 多ID
    w = f.Args["回溯期数"][0]
    return pd.DataFrame(Mid).rolling(window=w+1).mean().values[w:]
    
    
IDs = ["000001.SZ", "000002.SZ", "000003.SZ"]
DTs = FT.getDateTime(start_dt=dt.datetime(2018,11,5), end_dt=dt.datetime(2018,11,9))
DTRuler = FT.getDateTime(start_dt=dt.datetime(2018,10,29), end_dt=dt.datetime(2018,11,9))

Args = {
    "回溯期数": [2-1, 2-1],
    "回溯模式": ["滚动窗口", "滚动窗口"],
    "自身回溯期数": 0,
    "自身回溯模式": "滚动窗口",
    "自身初始值": None,
    "算子": TimeFun,
    "参数": {}, 
    "数据类型": "double",
    "运算时点": "多时点", 
    "运算ID": "多ID"
}
TimeFactor = QS.FactorDB.TimeOperation(name="TimeFactor", 
                                       descriptors=[High, Low], 
                                       sys_args=Args)


print(Args)
print("======================================================")
print(TimeFactor.readData(ids=IDs, dts=DTs, dt_ruler=DTRuler))

{'回溯期数': [1, 1], '回溯模式': ['滚动窗口', '滚动窗口'], '自身回溯期数': 0, '自身回溯模式': '滚动窗口', '自身初始值': None, '算子': <function TimeFun at 0x000002F57FA67AF0>, '参数': {}, '数据类型': 'double', '运算时点': '多时点', '运算ID': '多ID'}
idt : [datetime.datetime(2018, 11, 2, 0, 0), datetime.datetime(2018, 11, 5, 0, 0), datetime.datetime(2018, 11, 6, 0, 0), datetime.datetime(2018, 11, 7, 0, 0), datetime.datetime(2018, 11, 8, 0, 0), datetime.datetime(2018, 11, 9, 0, 0)],    iid : ['000001.SZ', '000002.SZ', '000003.SZ'],    args: {},    x : [array([[11.16, 25.25,   nan],
       [11.04, 24.48,   nan],
       [10.93, 24.4 ,   nan],
       [10.95, 24.5 ,   nan],
       [10.97, 24.59,   nan],
       [10.73, 24.33,   nan]]), array([[10.83, 24.  ,   nan],
       [10.82, 23.7 ,   nan],
       [10.73, 23.71,   nan],
       [10.75, 23.71,   nan],
       [10.82, 23.92,   nan],
       [10.4 , 23.53,   nan]])]
            000001.SZ  000002.SZ  000003.SZ
2018-11-05    10.9625    24.3575        NaN
2018-11-06    10.8800    24.0725        NaN
20

### 自身迭代, 滚动窗口模式

**自身迭代, 滚动窗口模式**: "自身回溯期数">0 (自身迭代), 所有描述子的 "回溯模式" 均为 "滚动窗口":
* "运算时点" 参数一般为 "单时点", "运算ID" 为 "单ID":
    * f: Factor, 该算子隶属于的因子对象, 即调用该算子的因子对象
    * idt: list(datetime), 当前回溯期数最大的因子(包括该因子自身)的时间序列
    * iid: str, 当前待计算的 ID
    * x: list(array), 描述子数据, 其中 x[0] 为自身因子在时点为 idt[-1] ID 为 iid 时的回溯数据, 在运算首个时点为 array(shape=(0, )), 其他时点为 array(shape=(自身回溯期数, )); x[i](i>=1): 第 i-1 个描述子在时点为 idt[-1] ID 为 iid 时的回溯数据, array(shape=(第 i-1 个描述子的回溯期数+1, )), +1 表示会把当前时点的数据也添加进去
    * args: dict, 用户传入的自定义参数
    * 返回值: 标量, 衍生因子值
* "运算时点" 参数一般为 "单时点", 如果 "运算ID" 为 "多ID":
    * f: Factor, 该算子隶属于的因子对象, 即调用该算子的因子对象
    * idt: list(datetime), 当前回溯期数最大的因子(包括该因子自身)的时间序列
    * iid: list(str), 当前一起计算的 ID 序列, 注意该序列并不一定为整个截面
    * x: list(array), 描述子数据, 其中 x[0] 为自身因子在时点为 idt[-1] 时的回溯数据, 在运算首个时点为 array(shape=(0, len(iid))), 其他时点为 array(shape=(自身回溯期数, len(iid))); x[i](i>=1): 第 i-1 个描述子在时点为 idt[-1] 时的回溯数据, array(shape=(第 i-1 个描述子的回溯期数+1, len(iid))), +1 表示会把当前时点的数据也添加进去
    * args: dict, 用户传入的自定义参数
    * 返回值: array(shape=(len(iid), )), 衍生因子值

In [5]:
# 时间序列运算: 自身迭代, 滚动窗口模式
FT = FDB.getTable(table_name="stock_cn_day_bar_nafilled")
High, Low = FT.getFactor("high"), FT.getFactor("low")

def TimeFun(f, idt, iid, x, args):
    print(f"idt : {idt},", f"iid : {iid},", f"args: {args},", f"x : {x}", sep="    ")
    print("======================================================")
    Mid = (x[1] + x[2]) / 2
    # 单时点
    return 0.5 * x[0][-1] + 0.5 * np.nanmean(Mid)
    
    
IDs = ["000001.SZ", "000002.SZ", "000003.SZ"]
DTs = FT.getDateTime(start_dt=dt.datetime(2018,11,5), end_dt=dt.datetime(2018,11,9))
DTRuler = FT.getDateTime(start_dt=dt.datetime(2018,10,29), end_dt=dt.datetime(2018,11,9))

Args = {
    "回溯期数": [2-1, 2-1],
    "回溯模式": ["滚动窗口", "滚动窗口"],
    "自身回溯期数": 1,
    "自身回溯模式": "滚动窗口",
    "自身初始值": pd.DataFrame(0, index=[dt.datetime(2018,11,2)], columns=IDs),
    "算子": TimeFun,
    "参数": {}, 
    "数据类型": "double",
    "运算时点": "单时点", 
    "运算ID": "多ID"
}
TimeFactor = QS.FactorDB.TimeOperation(name="TimeFactor", 
                                       descriptors=[High, Low], 
                                       sys_args=Args)


print(Args)
print("======================================================")
print(TimeFactor.readData(ids=IDs, dts=DTs, dt_ruler=DTRuler))

{'回溯期数': [1, 1], '回溯模式': ['滚动窗口', '滚动窗口'], '自身回溯期数': 1, '自身回溯模式': '滚动窗口', '自身初始值':             000001.SZ  000002.SZ  000003.SZ
2018-11-02          0          0          0, '算子': <function TimeFun at 0x000002F55C265820>, '参数': {}, '数据类型': 'double', '运算时点': '单时点', '运算ID': '多ID'}
idt : [datetime.datetime(2018, 11, 2, 0, 0), datetime.datetime(2018, 11, 5, 0, 0)],    iid : ['000001.SZ', '000002.SZ', '000003.SZ'],    args: {},    x : [array([[0., 0., 0.]]), array([[11.16, 25.25,   nan],
       [11.04, 24.48,   nan]]), array([[10.83, 24.  ,   nan],
       [10.82, 23.7 ,   nan]])]
idt : [datetime.datetime(2018, 11, 5, 0, 0), datetime.datetime(2018, 11, 6, 0, 0)],    iid : ['000001.SZ', '000002.SZ', '000003.SZ'],    args: {},    x : [array([[8.83, 8.83, 8.83]]), array([[11.04, 24.48,   nan],
       [10.93, 24.4 ,   nan]]), array([[10.82, 23.7 ,   nan],
       [10.73, 23.71,   nan]])]
idt : [datetime.datetime(2018, 11, 6, 0, 0), datetime.datetime(2018, 11, 7, 0, 0)],    iid : ['000001.SZ', '0000

### 无自身迭代, 扩张窗口模式

**无自身迭代, 扩张窗口模式**: "自身回溯期数" 为 0 (无自身迭代), 有描述子的 "回溯模式" 为 "扩张窗口":
* "运算时点" 为 "单时点", "运算ID" 为 "单ID":
    * f: Factor, 该算子隶属于的因子对象, 即调用该算子的因子对象
    * idt: list(datetime)
    * iid: str, 当前待计算的 ID
    * x: list(array), 描述子数据, x[i]: 第 i 个描述子在时点为 idt[-1] ID 为 iid 时的回溯数据, 如果第 i 个描述子的回溯模式为 "扩张窗口" 则为 array(shape=(第 i 个描述子的回溯期数+起始时点至当前时点 idt[-1] 的期数,)), 如果第 i 个描述子的回溯模式为 "滚动窗口" 则为 array(shape=(第 i 个描述子的回溯期数+1,)), +1 表示会把当前时点的数据也添加进去
    * args: dict, 用户传入的自定义参数
    * 返回值: 标量, 衍生因子值
* "运算时点" 为 "多时点", "运算ID" 为 "单ID":
    * f: Factor, 该算子隶属于的因子对象, 即调用该算子的因子对象
    * idt: list(datetime), 注意该序列并不一定为全体运算日期
    * iid: str, 当前待计算的 ID
    * x: list(array), 描述子数据, 其中, x[i]: 第 i 个描述子在 ID 为 iid 时的回溯数据, 如果第 i 个描述子的回溯模式为 "扩张窗口" 则为 array(shape=(第 i 个描述子的回溯期数+起始时点至 idt[-1] 的期数, )), 如果第 i 个描述子的回溯模式为 "滚动窗口" 则为 array(shape=(第 i 个描述子的回溯期数+待计算的时间序列长度, ))
    * args: dict, 用户传入的自定义参数
    * 返回值: array(shape=(待计算的时间序列长度, )), 衍生因子值
* "运算时点" 为 "单时点", "运算ID" 为 "多ID":
    * f: Factor, 该算子隶属于的因子对象, 即调用该算子的因子对象
    * idt: list(datetime)
    * iid: list(str), 当前一起计算的 ID 序列, 注意该序列并不一定为整个截面
    * x: list(array), 描述子数据, 其中, x[i]: 第 i 个描述子在时点为 idt[-1] 时的回溯数据, 如果第 i 个描述子的回溯模式为 "扩张窗口" 则为 array(shape=(第 i 个描述子的回溯期数+起始时点至 idt[-1] 的期数, len(iid))), 如果第 i 个描述子的回溯模式为 "滚动窗口" 则为 array(shape=(第 i 个描述子的回溯期数+1, len(iid))), +1 表示会把当前时点的数据也添加进去
    * args: dict, 用户传入的自定义参数
    * 返回值: array(shape=(len(iid), )), 衍生因子值
* "运算时点" 为 "多时点", "运算ID" 为 "多ID":
    * f: Factor, 该算子隶属于的因子对象, 即调用该算子的因子对象
    * idt: list(datetime), 注意该序列并不一定为全体运算日期
    * iid: list(str), 当前一起计算的 ID 序列, 注意该序列并不一定为整个截面
    * x: list(array), 描述子数据, 其中, x[i]: 第 i 个描述子数据, 如果第 i 个描述子的回溯模式为 "扩张窗口" 则为 array(shape=(第 i 个描述子的回溯期数+起始时点至 idt[-1] 的期数, len(iid))), 如果第 i 个描述子的回溯模式为 "滚动窗口" 则为 array(shape=(第 i 个描述子的回溯期数+待计算的时间序列长度, len(iid)))
    * args: dict, 用户传入的自定义参数
    * 返回值: array(shape=(待计算的时间序列长度, len(iid))), 衍生因子值

In [6]:
# 时间序列运算: 无自身迭代, 扩张窗口模式
FT = FDB.getTable(table_name="stock_cn_day_bar_nafilled")
High, Low = FT.getFactor("high"), FT.getFactor("low")

def TimeFun(f, idt, iid, x, args):
    print(f"idt : {idt},", f"iid : {iid},", f"args: {args},", f"x : {x}", sep="    ")
    print("======================================================")
    Mid = (x[0] + x[1]) / 2
    return np.nanmean(Mid)
    
    
IDs = ["000001.SZ", "000002.SZ", "000003.SZ"]
DTs = FT.getDateTime(start_dt=dt.datetime(2018,11,5), end_dt=dt.datetime(2018,11,9))
DTRuler = FT.getDateTime(start_dt=dt.datetime(2018,10,29), end_dt=dt.datetime(2018,11,9))

Args = {
    "回溯期数": [2-1, 2-1],
    "回溯模式": ["扩张窗口", "扩张窗口"],
    "自身回溯期数": 0,
    "自身回溯模式": "滚动窗口",
    "自身初始值": None,
    "算子": TimeFun,
    "参数": {}, 
    "数据类型": "double",
    "运算时点": "单时点", 
    "运算ID": "单ID"
}
TimeFactor = QS.FactorDB.TimeOperation(name="TimeFactor", 
                                       descriptors=[High, Low], 
                                       sys_args=Args)


print(Args)
print("======================================================")
print(TimeFactor.readData(ids=IDs, dts=DTs, dt_ruler=DTRuler))

{'回溯期数': [1, 1], '回溯模式': ['扩张窗口', '扩张窗口'], '自身回溯期数': 0, '自身回溯模式': '滚动窗口', '自身初始值': None, '算子': <function TimeFun at 0x000002F55C2659D0>, '参数': {}, '数据类型': 'double', '运算时点': '单时点', '运算ID': '单ID'}
idt : [datetime.datetime(2018, 11, 2, 0, 0), datetime.datetime(2018, 11, 5, 0, 0)],    iid : 000001.SZ,    args: {},    x : [array([11.16, 11.04]), array([10.83, 10.82])]
idt : [datetime.datetime(2018, 11, 2, 0, 0), datetime.datetime(2018, 11, 5, 0, 0)],    iid : 000002.SZ,    args: {},    x : [array([25.25, 24.48]), array([24. , 23.7])]
idt : [datetime.datetime(2018, 11, 2, 0, 0), datetime.datetime(2018, 11, 5, 0, 0)],    iid : 000003.SZ,    args: {},    x : [array([nan, nan]), array([nan, nan])]
idt : [datetime.datetime(2018, 11, 2, 0, 0), datetime.datetime(2018, 11, 5, 0, 0), datetime.datetime(2018, 11, 6, 0, 0)],    iid : 000001.SZ,    args: {},    x : [array([11.16, 11.04, 10.93]), array([10.83, 10.82, 10.73])]
idt : [datetime.datetime(2018, 11, 2, 0, 0), datetime.datetime(2018, 11, 5, 0, 

### 自身迭代, 扩张窗口模式

**自身迭代, 扩张窗口模式**: "自身回溯期数">0(自身迭代), 有描述子的回溯模式为 "扩张窗口":
* "运算时点" 参数一般为 "单时点", 如果 "运算ID" 为 "单ID":
    * f: Factor, 该算子隶属于的因子对象, 即调用该算子的因子对象
    * idt: list(datetime)
    * iid: str, 当前待计算的 ID
    * x: list(array), 描述子数据, 其中 x[0]: 自身因子数据, 在运算首个时点为 array(shape=(0, )), 其他时点为 array(shape=(自身回溯期数, )); x[i](i>=1): 第 i-1 个描述子数据, 如果第 i-1 个描述子的回溯模式为 “扩张窗口” 则为 array(shape=(第 i-1 个描述子的回溯期数+起始时点至当前时点 idt[-1] 的期数, )), 如果第 i-1 个描述子的回溯模式为 “滚动窗口” 则为 array(shape=(第 i-1 个描述子的回溯期数+1, )), +1 表示会把当前时点的数据也添加进去
    * args: dict, 用户传入的自定义参数
    * 返回值: 标量, 衍生因子值
* "运算时点" 参数一般为 "单时点", 如果 "运算ID" 为 "多ID":
    * f: Factor, 该算子隶属于的因子对象, 即调用该算子的因子对象
    * idt: list(datetime)
    * iid: list(str), 当前一起计算的 ID 序列, 注意该序列并不一定为整个截面
    * x: list(array), 描述子数据, 其中 x[0]: 自身因子数据, 在运算首个时点为 array(shape=(0, len(iid))), 其他时点为 array(shape=(自身回溯期数, len(iid))); x[i](i>=1): 第 i-1 个描述子数据, 如果第 i-1 个 描述子的回溯模式为 "扩张窗口" 则为 array(shape=(第 i-1 个描述子的回溯期数+起始时点至当前时点 idt[-1] 的期数, len(iid))), 如果第 i-1 个描述子的回溯模式为 "滚动窗口" 则为 array(shape=(第 i-1 个描述子的回溯期数+1, len(iid))), +1 表示会把当前时点的数据也添加进去
    * args: dict, 用户传入的自定义参数
    * 返回值: array(shape=(len(iid), )), 衍生因子值

In [7]:
# 时间序列运算: 自身迭代, 扩张窗口模式
FT = FDB.getTable(table_name="stock_cn_day_bar_nafilled")
High, Low = FT.getFactor("high"), FT.getFactor("low")

def TimeFun(f, idt, iid, x, args):
    print(f"idt : {idt},", f"iid : {iid},", f"args: {args},", f"x : {x}", sep="    ")
    print("======================================================")
    Mid = (x[1] + x[2]) / 2
    # 单时点
    return 0.5 * x[0][-1] + 0.5 * np.nanmean(Mid)
    
    
IDs = ["000001.SZ", "000002.SZ", "000003.SZ"]
DTs = FT.getDateTime(start_dt=dt.datetime(2018,11,5), end_dt=dt.datetime(2018,11,9))
DTRuler = FT.getDateTime(start_dt=dt.datetime(2018,10,29), end_dt=dt.datetime(2018,11,9))

Args = {
    "回溯期数": [2-1, 2-1],
    "回溯模式": ["扩张窗口", "扩张窗口"],
    "自身回溯期数": 1,
    "自身回溯模式": "滚动窗口",
    "自身初始值": pd.DataFrame(0, index=[dt.datetime(2018,11,2)], columns=IDs),
    "算子": TimeFun,
    "参数": {}, 
    "数据类型": "double",
    "运算时点": "单时点", 
    "运算ID": "单ID"
}
TimeFactor = QS.FactorDB.TimeOperation(name="TimeFactor", 
                                       descriptors=[High, Low], 
                                       sys_args=Args)


print(Args)
print("======================================================")
print(TimeFactor.readData(ids=IDs, dts=DTs, dt_ruler=DTRuler))

{'回溯期数': [1, 1], '回溯模式': ['扩张窗口', '扩张窗口'], '自身回溯期数': 1, '自身回溯模式': '滚动窗口', '自身初始值':             000001.SZ  000002.SZ  000003.SZ
2018-11-02          0          0          0, '算子': <function TimeFun at 0x000002F55C1EB160>, '参数': {}, '数据类型': 'double', '运算时点': '单时点', '运算ID': '单ID'}
idt : [datetime.datetime(2018, 11, 2, 0, 0), datetime.datetime(2018, 11, 5, 0, 0)],    iid : 000001.SZ,    args: {},    x : [array([0.]), array([11.16, 11.04]), array([10.83, 10.82])]
idt : [datetime.datetime(2018, 11, 2, 0, 0), datetime.datetime(2018, 11, 5, 0, 0)],    iid : 000002.SZ,    args: {},    x : [array([0.]), array([25.25, 24.48]), array([24. , 23.7])]
idt : [datetime.datetime(2018, 11, 2, 0, 0), datetime.datetime(2018, 11, 5, 0, 0)],    iid : 000003.SZ,    args: {},    x : [array([0.]), array([nan, nan]), array([nan, nan])]
idt : [datetime.datetime(2018, 11, 2, 0, 0), datetime.datetime(2018, 11, 5, 0, 0), datetime.datetime(2018, 11, 6, 0, 0)],    iid : 000001.SZ,    args: {},    x : [array([5.48125]),

## 横截面运算

固定时点不同因子间在横截面上的运算. 横截面运算的典型例子是各种因子数据标准化的处理. 比如 Z-score 标准化方法, 即是用每只证券的因子值减去整个截面因子值的平均数并处以截面标准差所得. 

横截面运算衍生因子由 SectionOperation 定义，其 __init__ 方法参数:
* name: str, 因子名称
* descriptors: list(Factor), 该因子依赖的因子列表
* sys_args: dict, 因子对象参数集

因子对象参数集:
* 算子: function, 用户自定义的运算函数, 输入参数必须为: (f, idt, iid, x, args)
* 参数: dict, 用户自定义的传递给算子的附加参数, 即算子的最后一个输入参数 args
* 数据类型: str, 可选 "double"(默认值), "string", "object", 因子的数据类型, 该参数也指明了算子的返回值类型
* 运算时点: str, 可选 "单时点"(默认值), "多时点", 传入算子进行运算的时点是单个时点还是一列时点, 通常可以在时间维度向量化运算的情况下选择 "多时点"
* 输出形式: str, 可选: "全截面"(默认值), "单ID", 算子返回值的数据类型, 如果是 "单ID", 则算子返回的是单个 ID 的因子数据; 如果是 "全截面", 则算子返回的是所有 ID 的因子数据.
* 描述子截面: list(None(默认值) 或者 list(str)), 每个描述子的 ID 序列, None 表示为全局 ID 序列

参数行为:
* "运算时点" 为 "单时点":
    * f:  Factor, 该算子隶属于的因子对象, 即调用该算子的因子对象
    * idt: datetime, 当前待计算的时点
    * iid: list(str), 注意该序列一定为整个截面
    * x: list(array(shape=(len(iid), ))), 描述子数据, x[i] 为第 i 个描述子在时点为 idt 时的截面数据
    * args: dict, 用户传入的自定义参数
    * 返回值: 如果输出形式为 "全截面" 则为 array(shape=(len(iid), )), 如果输出形式为 "单时点" 则为标量, 衍生因子值
* "运算时点" 为 "多时点":
    * f:  Factor, 该算子隶属于的因子对象, 即调用该算子的因子对象
    * idt: list(datetime), 注意该序列并不一定为全体运算时点
    * iid: list(str), 注意该序列一定为整个截面
    * x: list(array(shape=(len(idt), len(iid)))), 描述子数据, x[i] 为第 i 个描述子的因子数据
    * args: dict, 用户传入的自定义参数
    * 返回值: 如果输出形式为 "全截面" 则为 array(shape=(len(idt), len(iid))), 如果输出形式为 "单时点" 则为 array(shape=(len(idt),)), 衍生因子值

In [8]:
# 横截面运算
FT = FDB.getTable(table_name="stock_cn_day_bar_nafilled")
High, Low = FT.getFactor("high"), FT.getFactor("low")

def SectionFun(f, idt, iid, x, args):
    print(f"idt : {idt},", f"iid : {iid},", f"args: {args},", f"x : {x}", sep="    ")
    print("======================================================")
    Mid = (x[0] + x[1]) / 2
    # 单时点
    return (Mid - np.nanmean(Mid)) / np.nanstd(Mid)
    # 多时点
    # return ((Mid.T - np.nanmean(Mid, axis=1)) / np.nanstd(Mid, axis=1)).T

Args = {
    "描述子截面": [None, None], 
    "算子": SectionFun, 
    "参数": {}, 
    "数据类型": "double",
    "运算时点": "单时点", 
    "输出形式": "全截面"
}
SectionFactor = QS.FactorDB.SectionOperation(name="SectionFactor", 
                                             descriptors=[High, Low], 
                                             sys_args=Args)

IDs = ["000001.SZ", "000002.SZ", "000003.SZ"]
DTs = FT.getDateTime(start_dt=dt.datetime(2018,11,5), end_dt=dt.datetime(2018,11,9))
print(Args)
print("======================================================")
print(SectionFactor.readData(ids=IDs, dts=DTs))

{'描述子截面': [None, None], '算子': <function SectionFun at 0x000002F55C3A2E50>, '参数': {}, '数据类型': 'double', '运算时点': '单时点', '输出形式': '全截面'}
idt : 2018-11-05 00:00:00,    iid : ['000001.SZ', '000002.SZ', '000003.SZ'],    args: {},    x : [array([11.04, 24.48,   nan]), array([10.82, 23.7 ,   nan])]
idt : 2018-11-06 00:00:00,    iid : ['000001.SZ', '000002.SZ', '000003.SZ'],    args: {},    x : [array([10.93, 24.4 ,   nan]), array([10.73, 23.71,   nan])]
idt : 2018-11-07 00:00:00,    iid : ['000001.SZ', '000002.SZ', '000003.SZ'],    args: {},    x : [array([10.95, 24.5 ,   nan]), array([10.75, 23.71,   nan])]
idt : 2018-11-08 00:00:00,    iid : ['000001.SZ', '000002.SZ', '000003.SZ'],    args: {},    x : [array([10.97, 24.59,   nan]), array([10.82, 23.92,   nan])]
idt : 2018-11-09 00:00:00,    iid : ['000001.SZ', '000002.SZ', '000003.SZ'],    args: {},    x : [array([10.73, 24.33,   nan]), array([10.4 , 23.53,   nan])]
            000001.SZ  000002.SZ  000003.SZ
2018-11-05       -1.0        1.0 

## 面板运算

需要时间序列和整个截面数据的不同因子间的运算. 面板运算是这里最复杂的一种运算, 同时间序列运算一样也需要指定每个描述子的回溯期数, 系统传递给定义算子的描述子数据是一个二维的面板数据. 面板运算的一个典型例子是进行时间序列和截面的双重标准化.

因子对象参数:
* 算子: function, 用户自定义的运算函数, 输入参数必须为: (f, idt, iid, x, args)
* 参数: dict, 用户自定义的传递给算子的附加参数, 即算子的最后一个输入参数 args.
* 数据类型: str, 可选 "double"(默认值), "string", "object", 因子的数据类型, 该参数也指明了自定义算子的返回值类型
* 运算时点: str, 可选 "单时点"(默认值), "多时点", 传入算子进行运算的时点是单个时点还是一列时点, 通常可以在时间维度向量化运算的情况下选择 "多时点"
* 回溯期数: list(int>=0), 每个描述子的回溯期数(不包括当前所在的时点), 比如计算 5 日平均, 则回溯期数为 5-1=4
* 回溯模式: list(str, 可选 "滚动窗口"(默认值), "扩张窗口"), 每个描述子的回溯窗口模式
* 自身回溯期数: int>=0, 默认值 0, 该因子本身需要回溯的期数, 0 表示无自身迭代, 并忽略 "自身回溯模式" 和 "自身初始值" 参数
* 自身回溯模式: str, 可选 "滚动窗口"(默认值), "扩张窗口", 该因子本身的回溯窗口模式
* 自身初始值: DataFrame(index=[时点], columns=[ID]), 自身回溯的初始值
* 输出形式: str, 可选: "全截面"(默认值), "单ID", 算子返回值的数据类型, 如果是 "单ID", 则算子返回的是单个 ID 的因子数据; 如果是 "全截面", 则算子返回的是所有 ID 的因子数据.
* 描述子截面: list(None(默认值) 或者 list(str)), 每个描述子的 ID 序列, None 表示为全局 ID 序列

面板运算各个参数的含义, 算子的输入参数以及返回值可以参考 [时间序列运算](#时间序列运算) 和 [横截面运算](#横截面运算) 的说明, 这里不再赘述.

In [9]:
# 面板运算
FT = FDB.getTable(table_name="stock_cn_day_bar_nafilled")
High, Low = FT.getFactor("high"), FT.getFactor("low")

def PanelFun(f, idt, iid, x, args):
    print(f"idt : {idt},", f"iid : {iid},", f"args: {args},", f"x : {x}", sep="    ")
    print("======================================================")
    Mid = (x[0] + x[1]) / 2
    Tmp = (Mid[-1] - np.nanmean(Mid, axis=0)) / np.nanstd(Mid, axis=0)
    return (Tmp - np.nanmean(Tmp)) / np.nanstd(Tmp)

IDs = ["000001.SZ", "000002.SZ", "000003.SZ"]
DTs = FT.getDateTime(start_dt=dt.datetime(2018,11,5), end_dt=dt.datetime(2018,11,9))
DTRuler = FT.getDateTime(start_dt=dt.datetime(2018,10,29), end_dt=dt.datetime(2018,11,9))

Args = {
    "回溯期数": [3-1, 3-1], 
    "回溯模式": ["滚动窗口", "滚动窗口"],
    "自身回溯期数": 0,
    "自身回溯模式": "滚动窗口",
    "自身初始值": None,
    "描述子截面": [None, None], 
    "算子": PanelFun, 
    "参数": {}, 
    "数据类型": "double",
    "运算时点": "单时点", 
    "输出形式": "全截面"
}
PanelFactor = QS.FactorDB.PanelOperation(name="PanelFactor", 
                                         descriptors=[High, Low], 
                                         sys_args=Args)
print(Args)
print("======================================================")
print(PanelFactor.readData(ids=IDs, dts=DTs, dt_ruler=DTRuler))

{'回溯期数': [2, 2], '回溯模式': ['滚动窗口', '滚动窗口'], '自身回溯期数': 0, '自身回溯模式': '滚动窗口', '自身初始值': None, '描述子截面': [None, None], '算子': <function PanelFun at 0x000002F55C3A2D30>, '参数': {}, '数据类型': 'double', '运算时点': '单时点', '输出形式': '全截面'}
idt : [datetime.datetime(2018, 11, 1, 0, 0), datetime.datetime(2018, 11, 2, 0, 0), datetime.datetime(2018, 11, 5, 0, 0)],    iid : ['000001.SZ', '000002.SZ', '000003.SZ'],    args: {},    x : [array([[11.05, 25.27,   nan],
       [11.16, 25.25,   nan],
       [11.04, 24.48,   nan]]), array([[10.76, 24.28,   nan],
       [10.83, 24.  ,   nan],
       [10.82, 23.7 ,   nan]])]
idt : [datetime.datetime(2018, 11, 2, 0, 0), datetime.datetime(2018, 11, 5, 0, 0), datetime.datetime(2018, 11, 6, 0, 0)],    iid : ['000001.SZ', '000002.SZ', '000003.SZ'],    args: {},    x : [array([[11.16, 25.25,   nan],
       [11.04, 24.48,   nan],
       [10.93, 24.4 ,   nan]]), array([[10.83, 24.  ,   nan],
       [10.82, 23.7 ,   nan],
       [10.73, 23.71,   nan]])]
idt : [datetime.datetime(20

# 运算符重载

对于因子对象, 系统实现了大多数运算符的重载. 需要特别注意的是, 因子运算表达式定义的新因子没有因子名称, 是不完全的因子, 如果作为最终输出数据的因子必须调用 Factorize 函数将该未命名的因子对象因子化后才能成为完全的因子.

Factorize 函数的输入参数和返回值:
* factor_object：Factor, 因子对象
* factor_name: str, 指定的因子名称
* other_args: dict, 其他参数
* 返回值: Factor, 命名后的因子对象

重载的运算符如下所示(A, B, C, …表示因子对象, 或者具体的标量数据), 使用运算符构建表达式的输出结果本质上是基于单点运算构建的衍生因子:
* A + B: 将因子 A 数据和因子 B 数据对应相加, 返回新因子
* A - B: 将因子 A 数据和因子 B 数据对应相减, 返回新因子
* A * B: 将因子 A 数据和因子 B 数据对应相乘, 返回新因子
* A / B: 将因子 A 数据和因子 B 数据对应相除, 返回新因子
* A // B: 将因子 A 数据和因子 B 数据对应做向下取整除法, 返回新因子
* A % B: 将因子 A 数据和因子 B 数据对应取余, 返回新因子
* A ** B: 将因子 A 数据和因子 B 数据对应取乘方, A 是底数, B 是幂次, 返回新因子
* A < B: 将因子 A 数据和因子 B 数据对应取小于运算, 结果是 True 或者 False, 返回新因子
* A <= B: 将因子 A 数据和因子 B 数据对应取小于等于运算, 结果是 True 或者 False, 返回新因子
* A > B: 将因子 A 数据和因子 B 数据对应取大于运算, 结果是 True 或者 False, 返回新因子
* A >= B: 将因子 A 数据和因子 B 数据对应取大于等于运算, 结果是 True 或者 False, 返回新因子
* A == B: 将因子 A 数据和因子 B 数据对应是否相等, 结果是 True 或者 False, 返回新因子
* A != B: 将因子 A 数据和因子 B 数据对应是否不等, 结果是 True 或者 False, 返回新因子
* A & B: 将因子 A 数据和因子 B 数据对应做并操作, A 和 B 的数据必须为 True 或者 False的逻辑值, 返回新因子
* A | B: 将因子 A 数据和因子 B 数据对应做或操作, A 和 B 的数据必须为 True 或者 False的逻辑值, 返回新因子
* A ^ B: 将因子 A 数据和因子 B 数据对应做异或操作, A 和 B 的数据必须为 True 或者 False的逻辑值, 返回新因子
* ~A: 将因子 A 数据做取反操作, A 的数据必须为 True 或者 False的逻辑值, 返回新因子
* abs(A): 将因子 A 数据取绝对值, 返回新因子
* -A: 将因子 A 数据取相反数, 返回新因子

In [10]:
# 运算符重载
FT = FDB.getTable(table_name="stock_cn_day_bar_nafilled")
High, Low = FT.getFactor("high"), FT.getFactor("low")

Factorize = QS.FactorDB.Factorize
# Mid = (High + Low) / 2
Mid = Factorize((High + Low) / 2, factor_name="Mid")

IDs = ["000001.SZ", "000002.SZ", "000003.SZ"]
DTs = FT.getDateTime(start_dt=dt.datetime(2018,11,5), end_dt=dt.datetime(2018,11,9))
print("因子名称 : ", Mid.Name)
print(Mid.readData(ids=IDs, dts=DTs))

因子名称 :  Mid
            000001.SZ  000002.SZ  000003.SZ
2018-11-05     10.930     24.090        NaN
2018-11-06     10.830     24.055        NaN
2018-11-07     10.850     24.105        NaN
2018-11-08     10.895     24.255        NaN
2018-11-09     10.565     23.930        NaN


# 内置算子

内置算子定义在模块 FactorDataBase.FactorTools 中, 每个内置算子的其他参数 keywords 中可以设置 factor_name=因子名, 通过 factor_name 参数命名后的因子不需要再调用 Factorize 函数.

In [11]:
# 内置算子
FT = FDB.getTable(table_name="stock_cn_day_bar_nafilled")
High, Low = FT.getFactor("high"), FT.getFactor("low")

fd = QS.FactorDB.FactorTools
Mid = fd.nanmean(High, Low, factor_name="Mid")

IDs = ["000001.SZ", "000002.SZ", "000003.SZ"]
DTs = FT.getDateTime(start_dt=dt.datetime(2018,11,5), end_dt=dt.datetime(2018,11,9))
print("因子名称 : ", Mid.Name)
print(Mid.readData(ids=IDs, dts=DTs))

因子名称 :  Mid
            000001.SZ  000002.SZ  000003.SZ
2018-11-05     10.930     24.090        NaN
2018-11-06     10.830     24.055        NaN
2018-11-07     10.850     24.105        NaN
2018-11-08     10.895     24.255        NaN
2018-11-09     10.565     23.930        NaN


# 自定义因子表

继承自 FactorTable, 该因子表没有所属的因子库, 是由用户通过添加因子而自定义的因子表, 构造函数参数:
* name: str, 因子表名称.
* sys_args: 参见 [QS 对象](../基本框架.ipynb#QuantStudio-对象)
* config_file: 参见 [QS 对象](../基本框架.ipynb#QuantStudio-对象)

In [12]:
# 创建自定义因子表
CFT = QS.FactorDB.CustomFT(name="我的因子表")
print("自定义因子表 : ", CFT.Name)
print("参数集 : ", CFT.Args)

自定义因子表 :  我的因子表
参数集 :  {'批量模式': {'运算时点': [], '运算ID': [], '运算因子': [], '子进程数': 0, '时点标尺': [], '截面ID': []}, '遍历模式': {'向前缓冲时点数': 600, '向后缓冲时点数': 1, '缓冲模式': '因子', '最大缓冲因子数': 60, '最大缓冲ID数': 10000, '缓冲区大小': 300, '遍历时点': [], '遍历ID': []}}


## 添加因子

addFactors(factor_list=[], factor_table=None, factor_names=None, args={})

添加因子, 将 factor_list 列表中的因子以及因子表 factor_table 中的 factor_names 里列示的因子添加到自定义因子表中:	
* factor_list: list(Factor), 默认值 [], 因子对象列表
* factor_table: FactorTable, 默认值 None, 因子表对象
* factor_names: list(str), 默认值 [], 因子表 factor_table 中的因子名列表
* args: dict, 默认值 {}, 因子表 factor_table 调用 getFactor 获取因子对象时传入的参数集
* 返回: int, ErrorCode

In [13]:
# 添加因子
CFT.deleteFactors(factor_names=None)
FT = FDB.getTable(table_name="stock_cn_day_bar_nafilled")
High, Low = FT.getFactor("high"), FT.getFactor("low")
Mid = Factorize((High + Low) / 2, factor_name="mid")

CFT.addFactors(factor_list=[Mid, High, Low])
CFT.addFactors(factor_table=FT, factor_names=["open", "close"])
print(CFT.FactorNames)

['close', 'high', 'low', 'mid', 'open']


## 重命名因子

renameFactor(factor_name, new_factor_name)

重命名因子:
* factor_name: str, 旧因子名
* new_factor_name:, str, 新因子名
* 返回: int, ErrorCode

In [14]:
# 重命名因子
CFT.renameFactor(factor_name="open", new_factor_name="OPEN")
print(CFT.FactorNames)

['OPEN', 'close', 'high', 'low', 'mid']


## 删除因子

deleteFactors(factor_names=None)

删除因子:
* factor_names: list(str), 待删除的因子名列表, 为 None 表示删除所有因子
* 返回: int, ErrorCode

In [15]:
# 删除因子
CFT.deleteFactors(factor_names=["OPEN"])
print(CFT.FactorNames)

['close', 'high', 'low', 'mid']


## 设置时点序列

setDateTime(dts)

设置因子表的时间序列:
* dts: list(datetime.datetime), 时间序列
* 返回: int, ErrorCode

In [16]:
# 设置时点序列
FT = FDB.getTable(table_name="stock_cn_day_bar_nafilled")
DTs = FT.getDateTime(start_dt=dt.datetime(2018,11,5), end_dt=dt.datetime(2018,11,9))

CFT.setDateTime(dts=DTs)
print(CFT.getDateTime())

[datetime.datetime(2018, 11, 5, 0, 0), datetime.datetime(2018, 11, 6, 0, 0), datetime.datetime(2018, 11, 7, 0, 0), datetime.datetime(2018, 11, 8, 0, 0), datetime.datetime(2018, 11, 9, 0, 0)]


## 设置 ID 序列

setID(ids)

设置因子表的 ID 序列:
* ids: list(str), ID 序列
* 返回: int, ErrorCode

In [17]:
# 设置 ID 序列
IDs = ["000001.SZ", "000002.SZ", "000003.SZ"]

CFT.setID(ids=IDs)
print(CFT.getID())

['000001.SZ', '000002.SZ', '000003.SZ']


## 设置 ID 筛选条件

setIDFilter(id_filter_str)

设置因子表默认的 ID 筛选条件
* id_filter_str: str, ID 筛选条件, 比如 "(@close<=40) & (@mid>@close)"
* 返回: int, ErrorCode

In [18]:
# 设置 ID 筛选条件
IDs = ["000001.SZ", "000002.SZ", "000003.SZ"]

CFT.setIDFilter(id_filter_str="(@close>=10) & (@mid>@close)")
print(CFT.IDFilterStr)
print(CFT.getFilteredID(idt=dt.datetime(2018, 11, 5), id_filter_str=CFT.IDFilterStr))

(@close>=10) & (@mid>@close)
['000001.SZ', '000002.SZ']


## 读取数据

In [19]:
# 读取数据
FT = FDB.getTable(table_name="stock_cn_day_bar_nafilled")
DTs = FT.getDateTime(start_dt=dt.datetime(2018,11,15), end_dt=dt.datetime(2018,11,22))
IDs = ["000001.SZ", "000002.SZ", "000003.SZ"]

Data = CFT.readData(factor_names=["close", "mid"], ids=IDs, dts=DTs)
print(Data)

<class 'QuantStudio.Tools.QSObjects.Panel'>
Dimensions: 2 (items) x 6 (major_axis) x 3 (minor_axis)
Items axis: close to mid
Major_axis axis: 2018-11-15 00:00:00 to 2018-11-22 00:00:00
Minor_axis axis: 000001.SZ to 000003.SZ
